In [5]:
# Import libraries and connect to WRDS
import pandas as pd
import wrds

conn = wrds.Connection()

# Extract S&P 500 constituent data
sp500 = conn.raw_sql("""
    select a.*, b.date, b.ret
    from crsp.msp500list as a,
    crsp.msf as b
    where a.permno=b.permno
    and b.date >= a.start and b.date<= a.ending
    and b.date >= '01/01/2000'
    order by date;
""", date_cols=['start', 'ending', 'date'])

# Merge company names and basic info
mse = conn.raw_sql("""
    select comnam, ncusip, namedt, nameendt, 
    permno, shrcd, exchcd, hsiccd, ticker
    from crsp.msenames
""", date_cols=['namedt', 'nameendt'])
mse['nameendt'] = mse['nameendt'].fillna(pd.to_datetime('today'))

sp500_full = pd.merge(sp500, mse, how='left', on='permno')
sp500_full = sp500_full.loc[
    (sp500_full.date >= sp500_full.namedt) & 
    (sp500_full.date <= sp500_full.nameendt)
]

# Link with Compustat
ccm = conn.raw_sql("""
    select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
    linkdt, linkenddt
    from crsp.ccmxpf_linktable
    where substr(linktype,1,1)='L'
    and (linkprim ='C' or linkprim='P')
""", date_cols=['linkdt', 'linkenddt'])
ccm['linkenddt'] = ccm['linkenddt'].fillna(pd.to_datetime('today'))

sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])
sp500ccm = sp500ccm.loc[
    (sp500ccm['date'] >= sp500ccm['linkdt']) &
    (sp500ccm['date'] <= sp500ccm['linkenddt'])
]

# ----- Critical Addition: Get CIK from Compustat -----
compustat_company = conn.raw_sql("""
    select gvkey, cik
    from comp.company
""")

sp500ccm = pd.merge(
    sp500ccm, 
    compustat_company[['gvkey', 'cik']], 
    how='left', 
    on='gvkey'
)

# Filter 2020-2024 and add quarter grouping
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ticker', 'gvkey', 'cik']]
sp500ccm['date'] = pd.to_datetime(sp500ccm['date'])

sp500_2020_2024 = sp500ccm[
    (sp500ccm['date'] >= '2020-01-01') & 
    (sp500ccm['date'] <= '2024-12-31')
].copy()

sp500_2020_2024['year'] = sp500_2020_2024['date'].dt.year
sp500_2020_2024['quarter'] = sp500_2020_2024['date'].dt.to_period('Q')

# Export results
sp500_2020_2024.to_csv('sp500_tickers_2020_2024_by_quarter.csv', index=False)

Loading library list...
Done


In [6]:
# Read CSV file (assumes columns: ticker, quarter)
csv_path = "sp500_tickers_2020_2024_by_quarter.csv"
df_csv = pd.read_csv(csv_path)

# Read TXT file (tab-delimited with ticker-CIK mapping)
txt_path = "ticker.txt"
df_txt = pd.read_csv(txt_path, sep='\t', header=None, names=['ticker', 'cik'])

# Deduplicate: keep first CIK per ticker
df_txt_unique = df_txt.drop_duplicates(subset='ticker', keep='first')

# Merge data (left join to preserve all CSV records)
merged_df = pd.merge(
    df_csv[['ticker', 'quarter']],  # Maintain original quarter data
    df_txt_unique,
    on='ticker',
    how='left'
)

# Check for missing CIKs
missing_cik = merged_df[merged_df['cik'].isnull()]
if not missing_cik.empty:
    print("Tickers missing CIK:")
    print(missing_cik['ticker'].unique())

# Export results
merged_df.to_csv("merged_ticker_quarter_cik.csv", index=False)
print("Merge complete! Output: merged_ticker_quarter_cik.csv")

Merge complete! Output: merged_ticker_quarter_cik.csv
